## 利用GPT大型模型工具完成数据洞察
**本周以及下周的实验内容如下：**
请同学们参考《GPT数据科学系列课程实验手册》内容，并结合之前的作业经验，对GitHub上具有协作行为日志数据的500名用户的个人信息（包括姓名、公司、邮箱及其地理位置等）进行数据洞察分析。数据获取链接为：https://github.com/X-lab2017/dase-2024-autumn/tree/main/HomeWork/data/user_data

### 实验目标
- 培养数据处理与分析能力：通过实际操作，提升对大规模数据集的处理和分析能力。
- 掌握GPT工具的应用：学习如何利用GPT大型模型工具辅助完成数据洞察任务。
- 理解数据隐私与伦理：在处理包含个人信息的数据时，遵循数据隐私保护的原则和规范。
### 实验内容
1. 人口统计分析
- 国家和地区分布：统计用户所在国家和地区的分布，识别主要的开发者集中地。
- 城市级别分布：分析主要城市的开发者密度，发现技术热点区域。
- 时区分布:了解用户的时区分布，分析不同地区用户的协作时间模式。
2. 协作行为分析
- 提交频率：统计每个用户的提交次数，识别高活跃用户和低活跃用户。
3. 其他维度有趣的洞察（至少2个）
### 提交内容
- 数据分析代码。
- 最终的数据洞察报告（PDF格式）。